In [2]:
import numpy as np, pickle
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import featurizer, data_formatter, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import cosine_similarity

Using cuDNN version 5105 on context None
Mapped name None to device cuda: Tesla K40c (0000:02:00.0)


Loading model parameters...
Compiling encoders...
Loading tables...
Packing up...


Using TensorFlow backend.
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
with open('/data/jrgillick/speeches.pkl') as f:
    speeches = pickle.load(f)

In [4]:
with open('/data/jrgillick/individual_feature_lists_no_n_grams.pkl') as f:
    individual_feature_lists = pickle.load(f)

In [26]:
with open('/data/jrgillick/individual_feature_lists_skip_thoughts.pkl','wb') as f:
    pickle.dump(individual_feature_lists,f)

In [77]:
individual_feature_lists[0].keys()

['skip_thoughts',
 'euphony',
 'labels',
 'word_overlap',
 'phone',
 'substring',
 'liu',
 'bigrams',
 'cosine',
 'rst',
 'audio',
 'id']

In [4]:
#speeches = [speech.Speech(s.file_path.split('/')[1]) for s in tqdm(speeches)]

In [5]:
len(speeches)

310

In [7]:
import speech

In [302]:
for s in speeches:
    s.phrase_audio_features = s.get_phrase_audio_features()
    normalize_phrase_audio_features(s)

In [26]:
for s in speeches:
    try:
        s.normalize_phrase_audio_features()
    except:
        print s.file_path

In [8]:
s = speech.Speech('mike_pence_5')

In [276]:
reload(speech)

<module 'speech' from 'speech.py'>

In [10]:
s.normalize_phrase_audio_features()

In [27]:
s.phrase_audio_features

[{'internal_silence': -0.7186810197129201,
  'max_energy': 2.1579792,
  'max_pitch': 0.26451367,
  'mean_energy': 2.6789947,
  'mean_pitch': 0.6365634,
  'min_energy': 1.9573226,
  'min_pitch': 0.0,
  'range_energy': 1.8055665,
  'range_pitch': 0.26451367,
  'std_energy': 2.9101937,
  'std_pitch': 0.24777265},
 {'internal_silence': 1.0224972535110923,
  'max_energy': 0.5973549,
  'max_pitch': -0.54202604,
  'mean_energy': 1.5614538,
  'mean_pitch': -1.2314719,
  'min_energy': 1.5414388,
  'min_pitch': 0.0,
  'range_energy': 0.31531888,
  'range_pitch': -0.54202604,
  'std_energy': 0.23821816,
  'std_pitch': -0.8524239},
 {'internal_silence': 0.6170185974387099,
  'max_energy': 0.80153704,
  'max_pitch': 1.0411814,
  'mean_energy': 0.15645783,
  'mean_pitch': -0.44456735,
  'min_energy': -0.4916755,
  'min_pitch': 0.0,
  'range_energy': 0.89555174,
  'range_pitch': 1.0411814,
  'std_energy': 1.0221791,
  'std_pitch': 0.52062166},
 {'internal_silence': -1.7279407806485787,
  'max_energy'

In [92]:
speeches = [s for s in speeches if 'donald_trump_40' not in s.file_path]

In [28]:
len(speeches)

310

In [469]:
#featurizer.Featurizer(s).get_liu_feature_names()

In [39]:
len(all_train_t0_feats[0][0])

4352

In [90]:
reload(data_formatter)
reload(featurizer)

<module 'featurizer' from 'featurizer.py'>

In [7]:
train_speeches, test_speeches = train_test_split(speeches, test_size=0.2, random_state=43)

In [8]:
len(train_speeches)

248

In [9]:
len(test_speeches)

62

In [16]:
reload(models)

<module 'models' from 'models.pyc'>

In [393]:
len(speeches)

310

In [10]:
rst_0_feats = data_formatter.get_rst_data([speeches[0]])

100%|██████████| 1/1 [00:00<00:00, 16.72it/s]


In [11]:
len(rst_0_feats[0])

245

In [70]:
test_bigram_feats[0].shape

(244, 26704)

In [68]:
train_bigram_feats[0].shape

(107, 26704)

In [69]:
len(train_bigram_feats[1])

453

In [50]:
def normalize_data(X_train, X_test):
    new_X_train = np.copy(X_train)
    new_X_test = np.copy(X_test)
    for i in xrange(new_X_train.shape[1]):
        if set(new_X_train[:,i]) != set([0,1]) and set(new_X_train[:,i]) != set([0]) and set(new_X_train[:,i]) != set([1]):
            #print 'norming'
            #print set(new_X_train[:,i])
            mean = np.mean(new_X_train[:,i])
            new_X_train[:,i] -= mean
            new_X_test[:,i] -=mean
            std = np.std(X_train[:,i])
            if std > 1e-2:
                new_X_train[:,i] /= std
                new_X_test[:,i] /= std
    return new_X_train, new_X_test

In [402]:
reload(data_formatter)
reload(featurizer)

<module 'featurizer' from 'featurizer.py'>

In [20]:
train_bigram_feats = data_formatter.get_bigram_data(train_speeches)



  0%|          | 0/248 [00:00<?, ?it/s]

  0%|          | 1/248 [00:06<27:36,  6.70s/it]


  1%|          | 2/248 [00:29<47:32, 11.60s/it]

  1%|          | 3/248 [00:54<1:04:00, 15.67s/it]

  2%|▏         | 4/248 [01:04<56:10, 13.81s/it]  

  2%|▏         | 5/248 [01:23<1:02:16, 15.38s/it]

  2%|▏         | 6/248 [01:43<1:07:16, 16.68s/it]

  3%|▎         | 7/248 [01:53<59:27, 14.80s/it]  

  3%|▎         | 8/248 [02:12<1:03:47, 15.95s/it]

  4%|▎         | 9/248 [02:17<51:03, 12.82s/it]  

  4%|▍         | 10/248 [02:48<1:11:52, 18.12s/it]

  4%|▍         | 11/248 [03:06<1:12:13, 18.29s/it]

  5%|▍         | 12/248 [03:52<1:44:03, 26.46s/it]

  5%|▌         | 13/248 [04:33<2:01:13, 30.95s/it]

  6%|▌         | 14/248 [04:47<1:40:06, 25.67s/it]

  6%|▌         | 15/248 [05:15<1:43:14, 26.59s/it]

  6%|▋         | 16/248 [06:12<2:17:09, 35.47s/it]

  7%|▋         | 17/248 [06:47<2:16:14, 35.39s/it]

  7%|▋         | 18/248 [07:14<2:05:59, 32.87s/it]

  8%|▊         | 19/248 [07:18<1:

In [21]:
test_bigram_feats = data_formatter.get_bigram_data(test_speeches)

100%|██████████| 62/62 [19:28<00:00, 18.79s/it]


In [472]:
speech_names = [s.file_path for s in speeches]

In [39]:
for i in range(len(individual_feature_lists)):
    individual_feature_lists[i]['skip_thoughts'] = (train_skip_thought_feats + test_skip_thought_feats)[i]

In [76]:
for i in range(len(individual_feature_lists)):
    individual_feature_lists[i]['bigrams'] = (train_bigram_feats + test_bigram_feats)[i]

In [75]:
len(train_bigram_feats+test_bigram_feats)

310

In [40]:
len(train_skip_thought_feats + test_skip_thought_feats)

310

In [972]:
individual_feature_lists = []
for i in range(len(train_speeches)):
    h = {'id': train_speeches[i].file_path,
         'bigrams': train_bigram_feats[i],
         'rst': train_rst_feats[i],
         'audio': train_audio_feats[i],
         'euphony': train_euphony_feats[i],
         'liu': train_liu_feats[i],
         'cosine': np.array(train_cosine_feats[i]).reshape(-1,1),
         'phone': train_phone_feats[i],
         'substring': np.array(train_substring_feats[i]).reshape(-1,1),
         'word_overlap': np.array(train_word_overlap_feats[i]).reshape(-1,1),
         'labels': train_labels[i]}
    individual_feature_lists.append(h)
    
for i in range(len(test_speeches)):
    h = {'id': test_speeches[i].file_path,
         'bigrams': test_bigram_feats[i],
         'rst': test_rst_feats[i],
         'audio': test_audio_feats[i],
         'euphony': test_euphony_feats[i],
         'liu': test_liu_feats[i],
         'cosine': np.array(test_cosine_feats[i]).reshape(-1,1),
         'phone': test_phone_feats[i],
         'substring': np.array(test_substring_feats[i]).reshape(-1,1),
         'word_overlap': np.array(test_word_overlap_feats[i]).reshape(-1,1),
         'labels': test_labels[i]}
    individual_feature_lists.append(h)

In [974]:
"""
c = 0
for i in range(len(test_rst_feats)):
    for j in range(len(test_rst_feats[i])):
        for k in range(len(test_rst_feats[i][j])):
            if test_rst_feats[i][j][k] != test_rst_feats2[i][j][k]:
                test_rst_feats[i][j][k] = test_rst_feats2[i][j][k]
    

c = 0
for i in range(len(train_rst_feats)):
    for j in range(len(train_rst_feats[i])):
        for k in range(len(train_rst_feats[i][j])):
            if train_rst_feats[i][j][k] != train_rst_feats2[i][j][k]:
                train_rst_feats[i][j][k] = train_rst_feats2[i][j][k]
"""

'\nc = 0\nfor i in range(len(test_rst_feats)):\n    for j in range(len(test_rst_feats[i])):\n        for k in range(len(test_rst_feats[i][j])):\n            if test_rst_feats[i][j][k] != test_rst_feats2[i][j][k]:\n                test_rst_feats[i][j][k] = test_rst_feats2[i][j][k]\n    \n\nc = 0\nfor i in range(len(train_rst_feats)):\n    for j in range(len(train_rst_feats[i])):\n        for k in range(len(train_rst_feats[i][j])):\n            if train_rst_feats[i][j][k] != train_rst_feats2[i][j][k]:\n                train_rst_feats[i][j][k] = train_rst_feats2[i][j][k]\n'

In [975]:
#test_rst_feats_old = np.copy(test_rst_feats)
#train_rst_feats_old = np.copy(train_rst_feats)

In [1030]:
with open('/data/jrgillick/individual_feature_lists.pkl','wb') as f:
    pickle.dump(individual_feature_lists,f)

In [1031]:
5

5

In [29]:
def get_train_and_test_sets(speaker_name):
    train_set = []
    test_set = []
    for i in range(len(individual_feature_lists)):
        if speaker_name in individual_feature_lists[i]['id']:
            test_set.append(individual_feature_lists[i])
        else:
            train_set.append(individual_feature_lists[i])
    return train_set, test_set

In [35]:
def get_features(data_set, feature_names):
    all_feats = []
    for i in range(len(data_set)):
        feats = [data_set[i][name] for name in feature_names]
        all_feats.append(np.hstack(feats))
    return all_feats

def get_labels(data_set):
    all_labels = []
    for i in range(len(data_set)):
        all_labels.append(data_set[i]['labels'])
    return all_labels

In [90]:
def run_neural_cross_validation(feature_set, phrase_count):
    all_true = []; all_pred = []
    C = None
    for speaker in speaker_names:
        print speaker + ": "
        train_set, test_set = get_train_and_test_sets(speaker)

        labels_train = get_labels(train_set)
        data_train = get_features(train_set,feature_set)

        labels_test = get_labels(test_set)
        data_test = get_features(test_set,feature_set)

        current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(data_train,labels_train,phrase_count=phrase_count)
        current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(data_test,labels_test,phrase_count=phrase_count)

        X_train = np.array(current_train_formatted_feats).astype('float64')
        y_train = np.array(current_train_formatted_labs)
        
        print X_train.shape

        X_test = np.array(current_test_formatted_feats).astype('float64')
        y_test = np.array(current_test_formatted_labs)

        X_train, X_test = normalize_data(X_train,X_test)
        if C is None:
            model = models.train_cv_logistic_regression_faster(X_train,y_train)
            C = model.best_params_['C']
        else:
            model = models.train_logistic_regression(X_train,y_train,C)
        y_true, y_pred = models.evaluate_model(model, X_test, y_test)
        
        all_true += list(y_true)
        all_pred += list(y_pred)
        print
    return all_true, all_pred

In [91]:
reload(models)

<module 'models' from 'models.py'>

In [164]:
def run_logistic_cross_validation(feature_set, phrase_count, speaker_names=speaker_names, return_model=False):
    all_true = []; all_pred = []
    C = None
    for speaker in speaker_names:
        print speaker + ": "
        train_set, test_set = get_train_and_test_sets(speaker)

        labels_train = get_labels(train_set)
        data_train = get_features(train_set,feature_set)

        labels_test = get_labels(test_set)
        data_test = get_features(test_set,feature_set)

        current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(data_train,labels_train,phrase_count=phrase_count)
        current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(data_test,labels_test,phrase_count=phrase_count)

        X_train = np.array(current_train_formatted_feats).astype('float64')
        y_train = np.array(current_train_formatted_labs)
        
        print X_train.shape

        X_test = np.array(current_test_formatted_feats).astype('float64')
        y_test = np.array(current_test_formatted_labs)

        X_train, X_test = normalize_data(X_train,X_test)
        if C is None:
            model = models.train_cv_logistic_regression_faster(X_train,y_train)
            C = model.best_params_['C']
        else:
            model = models.train_logistic_regression(X_train,y_train,C)
        y_true, y_pred = models.evaluate_model(model, X_test, y_test)
        
        all_true += list(y_true)
        all_pred += list(y_pred)
        print
    if return_model:
        return all_true, all_pred,model
    else:
        return all_true, all_pred

In [94]:
reload(models)

<module 'models' from 'models.pyc'>

In [132]:
def run_logistic_single_speaker(feature_set, phrase_count, verbose=True):
    all_true = []; all_pred = []
    C = None
    for speaker in speaker_names:
        speaker_true = []; speaker_pred = []
        print speaker + ": "
        other, speaker_data = get_train_and_test_sets(speaker)
        speaker_data = np.array(speaker_data)
        #speaker_labels = get_labels(speaker_data)
        
        if len(speaker_data) == 1:
            continue
        
        if len(speaker_data) > 10:
            kf = KFold(n_splits=10)
        else:
            kf = KFold(n_splits=len(speaker_data))
            
        for train_index, test_index in kf.split(speaker_data):
            train_set = speaker_data[train_index]
            test_set = speaker_data[test_index]
            #train_set, test_set = train_test_split(speaker_data, test_size=0.2, random_state=43)

            labels_train = get_labels(train_set)
            data_train = get_features(train_set,feature_set)

            labels_test = get_labels(test_set)
            data_test = get_features(test_set,feature_set)

            current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(data_train,labels_train,phrase_count=phrase_count)
            current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(data_test,labels_test,phrase_count=phrase_count)

            X_train = np.array(current_train_formatted_feats).astype('float64')
            y_train = np.array(current_train_formatted_labs)

            X_test = np.array(current_test_formatted_feats).astype('float64')
            y_test = np.array(current_test_formatted_labs)

            #print X_train.shape
            #print X_test.shape

            if len(y_train) > 0 and len(y_test) > 0:

                X_train, X_test = normalize_data(X_train,X_test)
                if C is None and len(y_train) > 500:
                    model = models.train_cv_logistic_regression_faster(X_train,y_train)
                    C = model.best_params_['C']
                else:
                    model = models.train_logistic_regression(X_train,y_train)
                y_true, y_pred = models.evaluate_model(model, X_test, y_test, verbose=verbose)
                
                all_true += list(y_true)
                all_pred += list(y_pred)
                
                speaker_true += list(y_true)
                speaker_pred += list(y_pred)
                #print len(all_true)
            if verbose: print
        print_results(speaker_true, speaker_pred)
    #print len(all_true)
    #print len(all_pred)
    return all_true, all_pred

In [105]:
def print_results(total_true, total_pred):
    total_correct = np.sum(np.array(total_true) == np.array(total_pred))
    acc=float(total_correct) / len(total_true)
    std=math.sqrt( (acc * (1-acc)) / len(total_true) )
    precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
    #print "Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true))
    print "Accuracy: %.3f +/- %.3f (%s/%s) | Precision: %.3f | Recall: %.3f | F1: %.3f" % (acc, 1.96*std, total_correct, len(total_true), precision, recall, f1)


In [190]:
print_results(total_true, total_pred)

Accuracy: 0.667 +/- 0.101 (56/84) | Precision: 0.719 | Recall: 0.548 | F1: 0.622


In [191]:
feature_names_no_bigrams = np.array(list(featurizer.Featurizer(s).get_combined_feature_names()))

In [192]:
total_true, total_pred, full_model_no_bigrams = run_logistic_cross_validation(['rst','audio','euphony','liu','phone','substring','word_overlap','cosine'],phrase_count=1,speaker_names=['chris_christie'],return_model=True)

chris_christie: 
(35440, 155)
Accuracy: 0.631 +/- 0.103 (53/84) | Precision: 0.657 | Recall: 0.548 | F1: 0.597



In [193]:
coefs_no_bigrams = full_model_no_bigrams.best_estimator_.coef_[0]

In [195]:
zip(np.array(feature_names_no_bigrams)[np.argsort(coefs_no_bigrams)],coefs_no_bigrams[np.argsort(coefs_no_bigrams)])

[('LIWC_focuspast', -0.34171093585292117),
 ('LIWC_shehe', -0.27523560441910533),
 ('std_pitch', -0.24926773730194382),
 ('LIWC_function', -0.22805008698129714),
 ('LIWC_verb', -0.2157738621138877),
 ('LIWC_they', -0.17188635810268502),
 ('LIWC_tentat', -0.16138748712192302),
 ('min_phone_length', -0.15145283735017512),
 ('LIWC_death', -0.14578518613616417),
 ('LIWC_hear', -0.12908297096752105),
 ('LIWC_number', -0.12880780079300563),
 ('LIWC_adverb', -0.12824934364775303),
 ('rst_cat_N-reason', -0.10994142352162768),
 ('rst_cat_S-condition', -0.10016945333963785),
 ('max_phone_length', -0.08589100653229584),
 ('LIWC_feel', -0.08397120266221239),
 ('LIWC_see', -0.08302316835417352),
 ('range_energy', -0.08245363817827461),
 ('LIWC_home', -0.08197109402273431),
 ('LIWC_cogproc', -0.08032251931164111),
 ('rst_cat_S-reason', -0.07633149686550803),
 ('LIWC_ingest', -0.07461842301678845),
 ('LIWC_interrog', -0.06768763112927506),
 ('LIWC_insight', -0.06686435396564562),
 ('LIWC_quant', -0.0

In [184]:
total_true, total_pred, full_model = run_logistic_cross_validation(['rst','audio','euphony','liu','phone','substring','word_overlap','cosine','bigrams'],phrase_count=1,speaker_names=['chris_christie'],return_model=True)

chris_christie: 
(35440, 26859)
Accuracy: 0.667 +/- 0.101 (56/84) | Precision: 0.719 | Recall: 0.548 | F1: 0.622



In [185]:
coefs = full_model.best_estimator_.coef_[0]
len(coefs)

26859

In [181]:
feature_names = list(featurizer.Featurizer(s).get_combined_feature_names())
all_feature_names = np.array(feature_names + all_bigram_names)

In [183]:
all_feature_names.shape

(26859,)

In [189]:
zip(np.array(all_feature_names)[np.argsort(coefs)],coefs[np.argsort(coefs)])[-50:]

[('to_jerusalem', 0.42652114636265337),
 ('to_when', 0.4269395746192039),
 ('decided_not', 0.4298507177676792),
 ('do_some', 0.43104623161338007),
 ('of_country', 0.43317963584371155),
 ('arrested_and', 0.43827235724875313),
 ('reject_hate', 0.43838912603966773),
 ('wanting_to', 0.43859049388987953),
 ('to_inspire', 0.44032214342927334),
 ('pay_down', 0.44060447367761546),
 ('guys_i', 0.44068074139975555),
 ('ever_quit', 0.4445276696550155),
 ('dollars_and', 0.447649683115382),
 ("here's_my", 0.4529056004125084),
 ('chief_of', 0.45599242167397414),
 ('your_face', 0.4564571137830662),
 ('much_too', 0.4608782399454658),
 ('a_bright', 0.4611683586023265),
 ('your_employees', 0.4634115313089532),
 ("they're_moving", 0.46502935222319963),
 ('am_still', 0.46758046628816535),
 ('sometimes_you', 0.470884579840157),
 ('went_there', 0.4838791164013118),
 ('eyes_off', 0.49336406910006325),
 ('means_they', 0.49552497907807774),
 ('few_things', 0.49685127965115916),
 ('thank_you_feature', 0.4994892

In [201]:
len(feature_names)

155

In [206]:
print_results(total_true, total_pred)

Accuracy: 0.547 +/- 0.005 (19423/35490) | Precision: 0.558 | Recall: 0.454 | F1: 0.501


In [205]:
total_true, total_pred = run_logistic_single_speaker(['word_overlap','substring','rst'],phrase_count=1,verbose=False) #combined+ngram

donald_trump: 
Accuracy: 0.521 +/- 0.009 (6692/12846) | Precision: 0.527 | Recall: 0.402 | F1: 0.456
hilary_clinton: 
Accuracy: 0.555 +/- 0.011 (4027/7260) | Precision: 0.566 | Recall: 0.466 | F1: 0.511
bernie_sanders: 
Accuracy: 0.593 +/- 0.012 (3866/6518) | Precision: 0.611 | Recall: 0.513 | F1: 0.558
ted_cruz: 
Accuracy: 0.595 +/- 0.021 (1238/2082) | Precision: 0.622 | Recall: 0.483 | F1: 0.544
marco_rubio: 
Accuracy: 0.551 +/- 0.024 (874/1586) | Precision: 0.567 | Recall: 0.433 | F1: 0.491
john_kasich: 
Accuracy: 0.520 +/- 0.039 (332/638) | Precision: 0.524 | Recall: 0.448 | F1: 0.483
barack_obama: 
Accuracy: 0.512 +/- 0.023 (942/1840) | Precision: 0.513 | Recall: 0.479 | F1: 0.496
bill_clinton: 
Accuracy: 0.550 +/- 0.040 (321/584) | Precision: 0.566 | Recall: 0.428 | F1: 0.487
joe_biden: 
Accuracy: 0.537 +/- 0.042 (290/540) | Precision: 0.537 | Recall: 0.537 | F1: 0.537
mike_pence: 
Accuracy: 0.567 +/- 0.044 (279/492) | Precision: 0.560 | Recall: 0.622 | F1: 0.590
carly_fiorina: 


In [157]:
total_true, total_pred = run_logistic_single_speaker(['euphony','liu','audio','phone','word_overlap','substring','rst','cosine'],phrase_count=1,verbose=False) #combined+ngram

donald_trump: 
Accuracy: 0.633 +/- 0.008 (8128/12846) | Precision: 0.643 | Recall: 0.595 | F1: 0.618
hilary_clinton: 
Accuracy: 0.678 +/- 0.011 (4925/7260) | Precision: 0.682 | Recall: 0.668 | F1: 0.675
bernie_sanders: 
Accuracy: 0.675 +/- 0.011 (4400/6518) | Precision: 0.673 | Recall: 0.681 | F1: 0.677
ted_cruz: 
Accuracy: 0.627 +/- 0.021 (1306/2082) | Precision: 0.640 | Recall: 0.582 | F1: 0.610
marco_rubio: 
Accuracy: 0.604 +/- 0.024 (958/1586) | Precision: 0.612 | Recall: 0.567 | F1: 0.589
john_kasich: 
Accuracy: 0.679 +/- 0.036 (433/638) | Precision: 0.698 | Recall: 0.630 | F1: 0.662
barack_obama: 
Accuracy: 0.639 +/- 0.022 (1175/1840) | Precision: 0.641 | Recall: 0.630 | F1: 0.636
bill_clinton: 
Accuracy: 0.639 +/- 0.039 (373/584) | Precision: 0.646 | Recall: 0.613 | F1: 0.629
joe_biden: 
Accuracy: 0.609 +/- 0.041 (329/540) | Precision: 0.617 | Recall: 0.578 | F1: 0.597
mike_pence: 
Accuracy: 0.602 +/- 0.043 (296/492) | Precision: 0.589 | Recall: 0.671 | F1: 0.627
carly_fiorina: 

In [196]:
total_true1, total_pred1 = run_logistic_single_speaker(['bigrams'],phrase_count=1,verbose=False) #ngram

donald_trump: 
Accuracy: 0.607 +/- 0.008 (7798/12846) | Precision: 0.646 | Recall: 0.474 | F1: 0.547
hilary_clinton: 
Accuracy: 0.623 +/- 0.011 (4523/7260) | Precision: 0.666 | Recall: 0.494 | F1: 0.567
bernie_sanders: 
Accuracy: 0.706 +/- 0.011 (4600/6518) | Precision: 0.759 | Recall: 0.602 | F1: 0.672
ted_cruz: 
Accuracy: 0.697 +/- 0.020 (1452/2082) | Precision: 0.810 | Recall: 0.516 | F1: 0.630
marco_rubio: 
Accuracy: 0.620 +/- 0.024 (983/1586) | Precision: 0.671 | Recall: 0.469 | F1: 0.552
john_kasich: 
Accuracy: 0.580 +/- 0.038 (370/638) | Precision: 0.628 | Recall: 0.392 | F1: 0.483
barack_obama: 
Accuracy: 0.566 +/- 0.023 (1041/1840) | Precision: 0.599 | Recall: 0.398 | F1: 0.478
bill_clinton: 
Accuracy: 0.606 +/- 0.040 (354/584) | Precision: 0.667 | Recall: 0.425 | F1: 0.519
joe_biden: 
Accuracy: 0.604 +/- 0.041 (326/540) | Precision: 0.628 | Recall: 0.507 | F1: 0.561
mike_pence: 
Accuracy: 0.695 +/- 0.041 (342/492) | Precision: 0.843 | Recall: 0.480 | F1: 0.611
carly_fiorina: 

In [197]:
total_true2, total_pred2 = run_logistic_single_speaker(['euphony','liu','audio','bigrams'],phrase_count=1,verbose=False) #combined+ngram

donald_trump: 
Accuracy: 0.657 +/- 0.008 (8434/12846) | Precision: 0.674 | Recall: 0.606 | F1: 0.638
hilary_clinton: 
Accuracy: 0.679 +/- 0.011 (4931/7260) | Precision: 0.688 | Recall: 0.655 | F1: 0.671
bernie_sanders: 
Accuracy: 0.716 +/- 0.011 (4666/6518) | Precision: 0.724 | Recall: 0.699 | F1: 0.711
ted_cruz: 
Accuracy: 0.665 +/- 0.020 (1384/2082) | Precision: 0.685 | Recall: 0.610 | F1: 0.645
marco_rubio: 
Accuracy: 0.612 +/- 0.024 (971/1586) | Precision: 0.620 | Recall: 0.581 | F1: 0.600
john_kasich: 
Accuracy: 0.680 +/- 0.036 (434/638) | Precision: 0.691 | Recall: 0.652 | F1: 0.671
barack_obama: 
Accuracy: 0.659 +/- 0.022 (1212/1840) | Precision: 0.665 | Recall: 0.639 | F1: 0.652
bill_clinton: 
Accuracy: 0.635 +/- 0.039 (371/584) | Precision: 0.636 | Recall: 0.634 | F1: 0.635
joe_biden: 
Accuracy: 0.626 +/- 0.041 (338/540) | Precision: 0.645 | Recall: 0.559 | F1: 0.599
mike_pence: 
Accuracy: 0.681 +/- 0.041 (335/492) | Precision: 0.677 | Recall: 0.691 | F1: 0.684
carly_fiorina: 

In [198]:
total_true3, total_pred3 = run_logistic_single_speaker(['euphony','liu','audio','phone','word_overlap','substring','rst','bigrams'],phrase_count=1,verbose=False) #combined+ngram+tension

donald_trump: 
Accuracy: 0.659 +/- 0.008 (8460/12846) | Precision: 0.675 | Recall: 0.612 | F1: 0.642
hilary_clinton: 
Accuracy: 0.680 +/- 0.011 (4936/7260) | Precision: 0.690 | Recall: 0.653 | F1: 0.671
bernie_sanders: 
Accuracy: 0.717 +/- 0.011 (4675/6518) | Precision: 0.727 | Recall: 0.697 | F1: 0.711
ted_cruz: 
Accuracy: 0.671 +/- 0.020 (1397/2082) | Precision: 0.694 | Recall: 0.611 | F1: 0.650
marco_rubio: 
Accuracy: 0.619 +/- 0.024 (981/1586) | Precision: 0.632 | Recall: 0.569 | F1: 0.599
john_kasich: 
Accuracy: 0.679 +/- 0.036 (433/638) | Precision: 0.686 | Recall: 0.658 | F1: 0.672
barack_obama: 
Accuracy: 0.658 +/- 0.022 (1211/1840) | Precision: 0.666 | Recall: 0.634 | F1: 0.650
bill_clinton: 
Accuracy: 0.644 +/- 0.039 (376/584) | Precision: 0.669 | Recall: 0.568 | F1: 0.615
joe_biden: 
Accuracy: 0.565 +/- 0.042 (305/540) | Precision: 0.572 | Recall: 0.515 | F1: 0.542
mike_pence: 
Accuracy: 0.683 +/- 0.041 (336/492) | Precision: 0.679 | Recall: 0.695 | F1: 0.687
carly_fiorina: 

In [120]:
['skip_thoughts',
 'euphony',
 'labels',
 'word_overlap',
 'phone',
 'substring',
 'liu',
 'bigrams',
 'cosine',
 'rst',
 'audio',
 'id']


['skip_thoughts',
 'euphony',
 'labels',
 'word_overlap',
 'phone',
 'substring',
 'liu',
 'bigrams',
 'cosine',
 'rst',
 'audio',
 'id']

In [173]:
with open('/data/corpora/cspan/top_bigrams.pkl','rb') as f:
    top_bigrams = pickle.load(f)

In [174]:
all_bigram_names = [n[0] for n in top_bigrams]

In [177]:
#all_bigram_names

In [1156]:
all_true, all_pred = run_logistic_single_speaker(['bigrams','audio','liu','euphony','phone','cosine','substring','word_overlap','rst'],phrase_count=1)

donald_trump: 
Accuracy: 0.650 +/- 0.022 (1211/1862) | Precision: 0.677 | Recall: 0.576 | F1: 0.622

Accuracy: 0.670 +/- 0.027 (758/1132) | Precision: 0.709 | Recall: 0.576 | F1: 0.635

Accuracy: 0.659 +/- 0.027 (763/1158) | Precision: 0.685 | Recall: 0.587 | F1: 0.633

Accuracy: 0.648 +/- 0.028 (737/1138) | Precision: 0.662 | Recall: 0.605 | F1: 0.632

Accuracy: 0.676 +/- 0.022 (1144/1692) | Precision: 0.686 | Recall: 0.649 | F1: 0.667

Accuracy: 0.639 +/- 0.023 (1105/1730) | Precision: 0.668 | Recall: 0.553 | F1: 0.605

Accuracy: 0.639 +/- 0.027 (774/1212) | Precision: 0.644 | Recall: 0.619 | F1: 0.631

Accuracy: 0.670 +/- 0.033 (509/760) | Precision: 0.682 | Recall: 0.637 | F1: 0.659

Accuracy: 0.640 +/- 0.026 (857/1340) | Precision: 0.649 | Recall: 0.609 | F1: 0.628

Accuracy: 0.695 +/- 0.031 (571/822) | Precision: 0.711 | Recall: 0.657 | F1: 0.683

hilary_clinton: 
Accuracy: 0.676 +/- 0.033 (518/766) | Precision: 0.662 | Recall: 0.721 | F1: 0.690

Accuracy: 0.701 +/- 0.035 (467/66

In [1158]:
total_correct = np.sum(np.array(all_true) == np.array(all_pred))
acc=float(total_correct) / len(all_true)
std=math.sqrt( (acc * (1-acc)) / len(all_true) )
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(all_true, all_pred)]
print "Accuracy: %.3f +/- %.3f (%s/%s) | Precision: %.3f | Recall: %.3f | F1: %.3f" % (acc, 1.96*std, total_correct, len(all_true), precision, recall, f1)

Accuracy: 0.671 +/- 0.005 (23797/35490) | Precision: 0.683 | Recall: 0.635 | F1: 0.658


In [1150]:
total_words = 0
aligned_words = 0
for s in speeches:
    for word in s.alignment.words:
        total_words += 1
        if word['case'] == 'success':
            aligned_words += 1

In [1152]:
float(aligned_words)/total_words

0.8783685419891444

In [1154]:
total_words

1226829

In [1155]:
aligned_words

1077608

In [1143]:
s.alignment.words[55]

{u'case': u'not-found-in-audio',
 u'endOffset': 313,
 u'startOffset': 311,
 u'word': u'IN'}

In [1144]:
s.alignment.words[54]

{u'alignedWord': u'people',
 u'case': u'success',
 u'end': 2233.5099999999998,
 u'endOffset': 310,
 u'phones': [{u'duration': 0.07, u'phone': u'p_B'},
  {u'duration': 0.09, u'phone': u'iy_I'},
  {u'duration': 0.04, u'phone': u'p_I'},
  {u'duration': 0.06, u'phone': u'ah_I'},
  {u'duration': 0.03, u'phone': u'l_E'}],
 u'start': 2233.22,
 u'startOffset': 304,
 u'word': u'PEOPLE'}

In [1027]:
all_true, all_pred = run_logistic_cross_validation(['audio','liu','euphony','phone','cosine','substring','word_overlap','rst'],phrase_count=1)

donald_trump: 
(22678, 310)
Accuracy: 0.587 +/- 0.009 (7535/12846) | Precision: 0.596 | Recall: 0.536 | F1: 0.564

hilary_clinton: 
(28264, 310)
Accuracy: 0.659 +/- 0.011 (4783/7260) | Precision: 0.647 | Recall: 0.697 | F1: 0.671

bernie_sanders: 
(29006, 310)
Accuracy: 0.610 +/- 0.012 (3975/6518) | Precision: 0.632 | Recall: 0.525 | F1: 0.574

ted_cruz: 
(33442, 310)
Accuracy: 0.614 +/- 0.021 (1279/2082) | Precision: 0.637 | Recall: 0.530 | F1: 0.579

marco_rubio: 
(33938, 310)
Accuracy: 0.626 +/- 0.024 (993/1586) | Precision: 0.627 | Recall: 0.622 | F1: 0.624

john_kasich: 
(34886, 310)
Accuracy: 0.688 +/- 0.036 (439/638) | Precision: 0.699 | Recall: 0.661 | F1: 0.680

barack_obama: 
(33684, 310)
Accuracy: 0.620 +/- 0.022 (1141/1840) | Precision: 0.639 | Recall: 0.553 | F1: 0.593

bill_clinton: 
(34940, 310)
Accuracy: 0.618 +/- 0.039 (361/584) | Precision: 0.618 | Recall: 0.620 | F1: 0.619

joe_biden: 
(34984, 310)
Accuracy: 0.639 +/- 0.041 (345/540) | Precision: 0.652 | Recall: 0.59

In [1070]:
total_correct = np.sum(np.array(all_true) == np.array(all_pred))
acc=float(total_correct) / len(all_true)
std=math.sqrt( (acc * (1-acc)) / len(all_true) )
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(all_true, all_pred)]
print "Accuracy: %.3f +/- %.3f (%s/%s) | Precision: %.3f | Recall: %.3f | F1: %.3f" % (acc, 1.96*std, total_correct, len(all_true), precision, recall, f1)

Accuracy: 0.609 +/- 0.011 (4230/6942) | Precision: 0.614 | Recall: 0.590 | F1: 0.602


In [1075]:
len(all_pred)

6942

In [ ]:
#1 phrase, with deltas ['audio','liu','euphony','phone','cosine','substring','word_overlap','rst']
#Accuracy: 0.617 +/- 0.005 (21907/35524) | Precision: 0.625 | Recall: 0.582 | F1: 0.603

#1 phrase, no deltas ['audio','liu','euphony','phone','cosine','substring','word_overlap','rst']
#Accuracy: 0.616 +/- 0.005 (21871/35524) | Precision: 0.626 | Recall: 0.576 | F1: 0.600

#1 phrase, no deltas  ['audio','liu','euphony','phone']
#Accuracy: 0.615 +/- 0.005 (21852/35524) | Precision: 0.624 | Recall: 0.579 | F1: 0.601

#3 phrases, no deltas  ['audio','liu','euphony','phone']
#Accuracy: 0.618 +/- 0.005 (21684/35092) | Precision: 0.627 | Recall: 0.582 | F1: 0.604



In [993]:
total_correct

21569

In [54]:
from sklearn.metrics import precision_recall_fscore_support

In [1009]:
reload(models)

<module 'models' from 'models.py'>

In [805]:
type(np.array(test_rst_feats2)[0])

list

In [806]:
type(np.array(test_rst_feats)[0])

list

In [1004]:
model.best_params_['C']

0.01

In [997]:
reload(models)

<module 'models' from 'models.py'>

Accuracy: 0.551 +/- 0.009 (7079/12846) | Precision: 0.562 | Recall: 0.461 | F1: 0.507


(7079, 12846)

In [652]:
len(X_test)

12846

In [656]:
reload(models)

<module 'models' from 'models.py'>

In [ ]:
#train_rst_feats = data_formatter.get_rst_data(train_speeches)
#train_audio_feats = data_formatter.get_audio_data(train_speeches)
#train_labels = data_formatter.get_binary_labels(train_speeches)
#train_euphony_feats = data_formatter.get_euphony_data(train_speeches)
#train_liu_feats = data_formatter.get_liu_data(train_speeches)
#train_cosine_feats = data_formatter.get_vector_cosine_distance_data(train_speeches)
#train_phone_feats = data_formatter.get_phone_data(train_speeches)
#train_substring_feats = data_formatter.get_substring_data(train_speeches)
#train_word_overlap_feats = data_formatter.get_word_overlap_data(train_speeches)

In [699]:
train_rst_feats2 = data_formatter.get_rst_data(train_speeches)

100%|██████████| 248/248 [00:01<00:00, 148.27it/s]


In [700]:
test_rst_feats2 = data_formatter.get_rst_data(test_speeches)

100%|██████████| 62/62 [00:00<00:00, 128.87it/s]


In [737]:
len(train_rst_feats2[10])

259

In [738]:
len(train_rst_feats[10])

259

In [718]:
len(test_rst_feats[-1][0])

59

In [430]:
np.array(train_rst_feats[0]).shape

(107, 59)

In [457]:
test_bigram_feats = data_formatter.get_bigram_data(test_speeches[0:5])

100%|██████████| 5/5 [01:04<00:00, 14.48s/it]


In [462]:
test_bigram_feats[0].shape

(244, 26704)

In [454]:
np.nonzero(test_bigram_feats[0])

(array([  0,   0,   0, ..., 242, 243, 243]),
 array([2704, 3220, 4088, ..., 6587, 1226, 3232]))

In [434]:
test_unigram_feats[0].shape

(244, 4)

In [438]:
np.array(test_bigram_feats[0]).shape

(244,)

In [441]:
text_features.TextFeatures('this is a phrase').get_n_grams(2)

['this_is', 'is_a', 'a_phrase']

In [343]:
#train_unigram_feats = data_formatter.get_unigram_data(train_speeches)
train_rst_feats = data_formatter.get_rst_data(train_speeches)
train_audio_feats = data_formatter.get_audio_data(train_speeches)
train_labels = data_formatter.get_binary_labels(train_speeches)
train_euphony_feats = data_formatter.get_euphony_data(train_speeches)
train_liu_feats = data_formatter.get_liu_data(train_speeches)
train_cosine_feats = data_formatter.get_vector_cosine_distance_data(train_speeches)
train_phone_feats = data_formatter.get_phone_data(train_speeches)
train_substring_feats = data_formatter.get_substring_data(train_speeches)
train_word_overlap_feats = data_formatter.get_word_overlap_data(train_speeches)

100%|██████████| 248/248 [00:00<00:00, 281.08it/s]


In [344]:
#test_unigram_feats = data_formatter.get_unigram_data(test_speeches)
test_rst_feats = data_formatter.get_rst_data(test_speeches)
test_audio_feats = data_formatter.get_audio_data(test_speeches)
test_labels = data_formatter.get_binary_labels(test_speeches)
test_euphony_feats = data_formatter.get_euphony_data(test_speeches)
test_liu_feats = data_formatter.get_liu_data(test_speeches)
test_cosine_feats = data_formatter.get_vector_cosine_distance_data(test_speeches)
test_phone_feats = data_formatter.get_phone_data(test_speeches)
test_substring_feats = data_formatter.get_substring_data(test_speeches)
test_word_overlap_feats = data_formatter.get_word_overlap_data(test_speeches)

100%|██████████| 62/62 [00:00<00:00, 287.97it/s]


In [38]:
test_skip_thought_feats = data_formatter.get_skip_thought_data(test_speeches)

100%|██████████| 62/62 [05:38<00:00,  4.82s/it]


In [37]:
train_skip_thought_feats = data_formatter.get_skip_thought_data(train_speeches)

100%|██████████| 248/248 [23:03<00:00,  4.38s/it]


In [428]:
train_unigram_feats[0].shape

(453, 4259)

In [426]:
all_train_t0_feats =  [np.hstack([train_unigram_feats[i],train_audio_feats[i],train_euphony_feats[i],train_liu_feats[i],train_phone_feats[i]]) for i in range(len(train_audio_feats))]
all_test_t0_feats =  [np.hstack([test_unigram_feats[i],test_audio_feats[i],test_euphony_feats[i],test_liu_feats[i],test_phone_feats[i]]) for i in range(len(test_audio_feats))]

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [537]:
len(train_labels[0])

107

In [345]:
all_train_feats = [np.hstack([train_rst_feats[i],train_audio_feats[i],train_euphony_feats[i],train_liu_feats[i],train_phone_feats[i],train_substring_feats[i].reshape(-1,1),train_word_overlap_feats[i].reshape(-1,1),np.array(train_cosine_feats[i]).reshape(-1,1)]) for i in range(len(train_audio_feats))]
all_test_feats = [np.hstack([test_rst_feats[i],test_audio_feats[i],test_euphony_feats[i],test_liu_feats[i],test_phone_feats[i],test_substring_feats[i].reshape(-1,1),test_word_overlap_feats[i].reshape(-1,1),np.array(test_cosine_feats[i]).reshape(-1,1)]) for i in range(len(test_audio_feats))]

In [326]:
all_train_rst_feats = [np.hstack([train_rst_feats[i]]) for i in range(len(train_audio_feats))]
all_test_rst_feats = [np.hstack([test_rst_feats[i]]) for i in range(len(test_audio_feats))]

In [340]:
train_rst_formatted_feats, train_rst_formatted_labs = models.format_balanced_multiple_phrase_input(all_train_rst_feats,train_labels,phrase_count=1)
test_rst_formatted_feats, test_rst_formatted_labs = models.format_balanced_multiple_phrase_input(all_test_rst_feats,test_labels,phrase_count=1)

Empty...18
Empty...28
Empty...36
Empty...157


In [341]:
X_train, y_train = np.array(train_rst_formatted_feats), np.array(train_rst_formatted_labs)
X_test, y_test = np.array(test_rst_formatted_feats), np.array(test_rst_formatted_labs)

In [158]:
#train_formatted_feats_t0, train_formatted_labs_t0 = models.format_multiple_phrase_input(all_train_t0_feats,train_labels,phrase_count=1)
#test_formatted_feats_t0, test_formatted_labs_t0 = models.format_multiple_phrase_input(all_test_t0_feats,test_labels,phrase_count=1)

In [335]:
type(train_rst_feats[0])

list

In [336]:
featurizer.Featurizer(s).get_rst_feature_names()

['rst_cat_N-antithesis',
 'rst_cat_N-attribution',
 'rst_cat_N-circumstance',
 'rst_cat_N-comment',
 'rst_cat_N-comparison',
 'rst_cat_N-concession',
 'rst_cat_N-condition',
 'rst_cat_N-consequence',
 'rst_cat_N-contrast',
 'rst_cat_N-definition',
 'rst_cat_N-disjunction',
 'rst_cat_N-elaboration',
 'rst_cat_N-evidence',
 'rst_cat_N-example',
 'rst_cat_N-explanation',
 'rst_cat_N-hypothetical',
 'rst_cat_N-list',
 'rst_cat_N-manner',
 'rst_cat_N-means',
 'rst_cat_N-purpose',
 'rst_cat_N-question',
 'rst_cat_N-reason',
 'rst_cat_N-restatement',
 'rst_cat_N-result',
 'rst_cat_N-same_unit',
 'rst_cat_N-sequence',
 'rst_cat_N-temporal',
 'rst_cat_N-textualorganization',
 'rst_cat_N-topic',
 'rst_cat_S-antithesis',
 'rst_cat_S-attribution',
 'rst_cat_S-circumstance',
 'rst_cat_S-comment',
 'rst_cat_S-comparison',
 'rst_cat_S-concession',
 'rst_cat_S-condition',
 'rst_cat_S-consequence',
 'rst_cat_S-definition',
 'rst_cat_S-elaboration',
 'rst_cat_S-example',
 'rst_cat_S-explanation',
 'rst_

In [8]:
#unigram_list = featurizer.Featurizer(s).get_unigram_features()

In [9]:
#sparse.dok_matrix(np.array(unigram_list).shape)

In [346]:
train_formatted_feats, train_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(all_train_feats,train_labels,phrase_count=5)
test_formatted_feats, test_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(all_test_feats,test_labels,phrase_count=5)

Empty...18
Empty...28
Empty...36
Empty...157


In [347]:
X_train, y_train = np.array(train_formatted_feats), np.array(train_formatted_labs)
X_test, y_test = np.array(test_formatted_feats), np.array(test_formatted_labs)

In [10]:
X_train, y_train = np.array(train_formatted_feats_t0), np.array(train_formatted_labs_t0)
X_test, y_test = np.array(test_formatted_feats_t0), np.array(test_formatted_labs_t0)

In [167]:
X_train.shape

(28132, 1550)

In [168]:
X_test.shape

(6514, 1550)

In [348]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [349]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [350]:
with open('/data/jrgillick/x_train.pkl','wb') as f:
    pickle.dump(X_train,f)
    
with open('/data/jrgillick/x_test.pkl','wb') as f:
    pickle.dump(X_test,f)

with open('/data/jrgillick/y_train.pkl','wb') as f:
    pickle.dump(y_train,f)
    
with open('/data/jrgillick/y_test.pkl','wb') as f:
    pickle.dump(y_test,f)

In [352]:
s = speeches[221]
s.file_path
s.phrase_audio_features[19]

{'internal_silence': 0.3643679755792979,
 'max_energy': 0.27192634,
 'max_pitch': 172.26562,
 'mean_energy': 0.084792912,
 'mean_pitch': 44.370567,
 'min_energy': 0.0015051069,
 'min_pitch': -1.0,
 'range_energy': 0.27042124,
 'range_pitch': 173.26562,
 'std_energy': 0.057384759,
 'std_pitch': 42.044147}

In [356]:
file_paths = [s.file_path.split('/')[0] for s in speeches]
counts = {}
for p in file_paths:
    if p in counts:
        counts[p] +=1 
    else:
        counts[p] = 1

In [362]:
keys = counts.keys()
values = counts.values()
order = np.argsort(values)

In [474]:
l = zip(list(np.array(keys)[order]),list(np.array(values)[order]))
l.reverse()
l

[('donald_trump', 86),
 ('hilary_clinton', 72),
 ('bernie_sanders', 40),
 ('ted_cruz', 23),
 ('marco_rubio', 20),
 ('john_kasich', 17),
 ('barack_obama', 10),
 ('bill_clinton', 8),
 ('joe_biden', 7),
 ('mike_pence', 6),
 ('carly_fiorina', 5),
 ('jeb_bush', 5),
 ('rand_paul', 4),
 ('gary_johnson', 3),
 ('chris_christie', 3),
 ('rick_santorum', 1)]

In [479]:
speaker_names = [i[0] for i in l]
speaker_names

['donald_trump',
 'hilary_clinton',
 'bernie_sanders',
 'ted_cruz',
 'marco_rubio',
 'john_kasich',
 'barack_obama',
 'bill_clinton',
 'joe_biden',
 'mike_pence',
 'carly_fiorina',
 'jeb_bush',
 'rand_paul',
 'gary_johnson',
 'chris_christie',
 'rick_santorum']

In [387]:
total_u = 0
total_c = 0
for k in keys:
    k_speeches = [s for s in speeches if k in s.file_path]
    num_utterances = 0
    num_claps = 0
    for s in k_speeches:
        times = s.alignment.get_phrase_times()
        binary_labels = [int(s.alignment.applause_follows(t[1])) for t in times]
        num_utterances += (len(times))
        num_claps += np.sum(binary_labels)
    print k + " %d %d %f" % (num_utterances, num_claps, float(num_claps)/num_utterances)
    total_u += num_utterances
    total_c += num_claps

jeb_bush 1482 191 0.128880
joe_biden 1847 270 0.146183
hilary_clinton 12825 3933 0.306667
bernie_sanders 10994 3529 0.320993
rand_paul 939 134 0.142705
carly_fiorina 1222 129 0.105565
bill_clinton 2087 292 0.139914
chris_christie 1868 42 0.022484
mike_pence 1302 246 0.188940
donald_trump 27493 7357 0.267595
marco_rubio 4407 797 0.180849
gary_johnson 354 56 0.158192
john_kasich 4023 319 0.079294
rick_santorum 245 17 0.069388
ted_cruz 5873 1041 0.177252
barack_obama 3888 920 0.236626


In [388]:
total_u

80849

In [391]:
float(total_c) / 310

62.170967741935485

In [34]:
speaker_names = ['donald_trump',
 'hilary_clinton',
 'bernie_sanders',
 'ted_cruz',
 'marco_rubio',
 'john_kasich',
 'barack_obama',
 'bill_clinton',
 'joe_biden',
 'mike_pence',
 'carly_fiorina',
 'jeb_bush',
 'rand_paul',
 'gary_johnson',
 'chris_christie',
 'rick_santorum']

'barack_obama/barack_obama_2'

In [366]:
np.array(values)[order]

array([ 1,  3,  3,  4,  5,  5,  6,  7,  8, 10, 17, 20, 23, 40, 72, 86])

In [360]:
[(k,counts[k]) for k in keys]

[('jeb_bush', 5),
 ('joe_biden', 7),
 ('hilary_clinton', 72),
 ('bernie_sanders', 40),
 ('rand_paul', 4),
 ('carly_fiorina', 5),
 ('bill_clinton', 8),
 ('chris_christie', 3),
 ('mike_pence', 6),
 ('donald_trump', 86),
 ('marco_rubio', 20),
 ('gary_johnson', 3),
 ('john_kasich', 17),
 ('rick_santorum', 1),
 ('ted_cruz', 23),
 ('barack_obama', 10)]

In [217]:
phrase_audio_feats = s.get_phrase_audio_features()

In [220]:
phrase_audio_feats[10]

{'internal_silence': -1.0693530204840014,
 'max_energy': -0.27726322,
 'max_pitch': -0.16259487,
 'mean_energy': -0.027933361,
 'mean_pitch': 0.39010763,
 'min_energy': 0.21034582,
 'min_pitch': 0.0,
 'range_energy': -0.29338768,
 'range_pitch': -0.16259487,
 'std_energy': -0.81988597,
 'std_pitch': -0.82883531}

In [219]:
cats = {}
for k in phrase_audio_feats[0].keys():
    cats[k] = []
for f in phrase_audio_feats:
    for c in cats:
        cats[c].append(f[c])
means = {}
devs = {}
for k in cats.keys():
    means[k] = np.mean(cats[k])
    devs[k] = np.std(cats[k])
    
for f in phrase_audio_feats:
    for k in cats.keys():
        f[k] -= means[k]
        if devs[k] != 0:
            f[k] /= devs[k]

In [233]:
reload(speech)
s= speech.Speech('hilary_clinton_94')

In [236]:
s.phrase_audio_features

[{'internal_silence': 1.0252379059947219,
  'max_energy': -1.4680401,
  'max_pitch': -2.1974638,
  'mean_energy': 1.0324765,
  'mean_pitch': -1.5669148,
  'min_energy': 1.9720827,
  'min_pitch': 0.0,
  'range_energy': -1.6798016,
  'range_pitch': -2.1974638,
  'std_energy': -1.4055945,
  'std_pitch': -1.7920786},
 {'internal_silence': 0.88352131117379873,
  'max_energy': -0.28310025,
  'max_pitch': 1.0879167,
  'mean_energy': 1.8735211,
  'mean_pitch': -0.52128232,
  'min_energy': 3.4222355,
  'min_pitch': 0.0,
  'range_energy': -0.77184236,
  'range_pitch': 1.0879167,
  'std_energy': -1.3194878,
  'std_pitch': 0.94687879},
 {'internal_silence': -0.063377201046082146,
  'max_energy': 0.30285221,
  'max_pitch': 0.72701865,
  'mean_energy': 0.66706252,
  'mean_pitch': 0.2286,
  'min_energy': -0.66714555,
  'min_pitch': 0.0,
  'range_energy': 0.38486695,
  'range_pitch': 0.72701865,
  'std_energy': 0.24091077,
  'std_pitch': 0.97134107},
 {'internal_silence': -1.4245705750674009,
  'max_e

In [234]:
s.phrase_audio_features

[{'internal_silence': 0.56164383561643838,
  'max_energy': 0.16206369,
  'max_pitch': 52.751198,
  'mean_energy': 0.10038508,
  'mean_pitch': 19.53491,
  'min_energy': 0.021743808,
  'min_pitch': -1.0,
  'range_energy': 0.14031988,
  'range_pitch': 53.751198,
  'std_energy': 0.038469348,
  'std_pitch': 23.380373},
 {'internal_silence': 0.53944020356234101,
  'max_energy': 0.22135666,
  'max_pitch': 190.08621,
  'mean_energy': 0.11274809,
  'mean_pitch': 33.715446,
  'min_energy': 0.033032931,
  'min_pitch': -1.0,
  'range_energy': 0.18832374,
  'range_pitch': 191.08621,
  'std_energy': 0.039254308,
  'std_pitch': 44.591888},
 {'internal_silence': 0.39108364599426176,
  'max_energy': 0.25067702,
  'max_pitch': 175.0,
  'mean_energy': 0.095013641,
  'mean_pitch': 43.885113,
  'min_energy': 0.0011979881,
  'min_pitch': -1.0,
  'range_energy': 0.24947903,
  'range_pitch': 176.0,
  'std_energy': 0.053479105,
  'std_pitch': 44.781334},
 {'internal_silence': 0.17781690140845072,
  'max_energy

In [235]:
s.normalize_phrase_audio_features()

In [214]:
devs['min_pitch'] ==0

True

In [171]:
with open('/data/jrgillick/x_train_deltas.pkl','wb') as f:
    pickle.dump(X_train,f)
    
with open('/data/jrgillick/x_test_deltas.pkl','wb') as f:
    pickle.dump(X_test,f)

with open('/data/jrgillick/y_train_deltas.pkl','wb') as f:
    pickle.dump(y_train,f)
    
with open('/data/jrgillick/y_test_deltas.pkl','wb') as f:
    pickle.dump(y_test,f)

In [217]:
#model = models.train_cv_logistic_regression(X_train,y_train,penalty='l1')

In [15]:
with open('/data/jrgillick/log_reg_model.pkl','rb') as f:
    model = pickle.load(f)

In [42]:
# 3 phrases with deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.657 +/- 0.011 (5014/7633) | Precision: 0.672 | Recall: 0.637 | F1: 0.654


(0.6568845801126687,
 0.67209997283346912,
 0.63664436438497174,
 0.65389189903528477)

In [167]:
# 5 phrases with deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.665 +/- 0.011 (5009/7534) | Precision: 0.665 | Recall: 0.652 | F1: 0.658


(0.6648526679054951,
 0.66529942575881873,
 0.65158007498660953,
 0.65836828575294271)

In [177]:
# 5 phrases with deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.663 +/- 0.011 (4998/7534) | Precision: 0.669 | Recall: 0.635 | F1: 0.652


(0.6633926201221131,
 0.66901805869074493,
 0.63497589716122116,
 0.65155262434734806)

In [147]:
feature_names = np.array(list(featurizer.Featurizer(s).get_combined_feature_names()))# + ['vec_' + str(i) for i in range(4800)])
print len(feature_names)
all_feature_names = []
for i in range(3):
    for prefix in ['t','delta']:
        for f in feature_names:
            all_feature_names.append(prefix+'_'+str(i)+'_'+f)
len(all_feature_names)

155


930

In [164]:
with open('all_feature_names.pkl','wb') as f:
    pickle.dump(all_feature_names,f)

In [144]:
with open('all_feature_names.pkl','rb') as f:
    all_feature_names = pickle.load(f)

In [150]:
feature_names

array(['rst_cat_N-antithesis', 'rst_cat_N-attribution',
       'rst_cat_N-circumstance', 'rst_cat_N-comment',
       'rst_cat_N-comparison', 'rst_cat_N-concession',
       'rst_cat_N-condition', 'rst_cat_N-consequence',
       'rst_cat_N-contrast', 'rst_cat_N-definition',
       'rst_cat_N-disjunction', 'rst_cat_N-elaboration',
       'rst_cat_N-evidence', 'rst_cat_N-example', 'rst_cat_N-explanation',
       'rst_cat_N-hypothetical', 'rst_cat_N-list', 'rst_cat_N-manner',
       'rst_cat_N-means', 'rst_cat_N-purpose', 'rst_cat_N-question',
       'rst_cat_N-reason', 'rst_cat_N-restatement', 'rst_cat_N-result',
       'rst_cat_N-same_unit', 'rst_cat_N-sequence', 'rst_cat_N-temporal',
       'rst_cat_N-textualorganization', 'rst_cat_N-topic',
       'rst_cat_S-antithesis', 'rst_cat_S-attribution',
       'rst_cat_S-circumstance', 'rst_cat_S-comment',
       'rst_cat_S-comparison', 'rst_cat_S-concession',
       'rst_cat_S-condition', 'rst_cat_S-consequence',
       'rst_cat_S-definition',

In [178]:
coefs = model.best_estimator_.coef_[0]

In [169]:
len(coefs)

940

In [179]:
zip(np.array(all_feature_names)[np.argsort(coefs)],coefs[np.argsort(coefs)])

[('t_0_std_pitch', -0.19083710199216203),
 ('t_0_min_phone_length', -0.17395552764903666),
 ('t_0_mean_pitch', -0.11530156278929017),
 ('t_0_LIWC_focuspast', -0.1129578905846733),
 ('t_0_range_energy', -0.081952255753081224),
 ('t_1_max_energy', -0.07173065667961652),
 ('t_0_LIWC_function', -0.06703763380566477),
 ('t_0_LIWC_tentat', -0.063148619071874551),
 ('t_1_LIWC_focuspast', -0.058994597390279221),
 ('t_0_LIWC_shehe', -0.052575618144318888),
 ('delta_0_LIWC_body', -0.05044552552647668),
 ('delta_0_min_energy', -0.049535263686345921),
 ('t_2_min_phone_length', -0.049503114718906858),
 ('t_0_LIWC_they', -0.040985158127851001),
 ('t_1_min_phone_length', -0.040576744281923566),
 ('t_4_mean_energy', -0.039017391089718717),
 ('t_3_mean_energy', -0.038743063629010116),
 ('t_1_std_pitch', -0.034930645003831348),
 ('t_0_LIWC_adverb', -0.034442550014040785),
 ('t_0_max_phone_length', -0.034181674107013713),
 ('t_2_LIWC_cause', -0.03171745920187747),
 ('t_4_LIWC_insight', -0.029943111713562

In [182]:
s = speeches[191]
s.file_path

'marco_rubio/marco_rubio_16'

In [184]:
s.alignment.get_phrase_text()[1]

u'GUYS HEAR'

In [33]:
phrases = s.alignment.get_phrase_text()
s_labels = featurizer.Featurizer(s).get_binary_labels()
times = s.alignment.get_phrase_times()
feature_list = featurizer.Featurizer(s).get_all_features()

In [34]:
s_feats, s_labs = models.format_multiple_phrase_input_with_deltas([feature_list],[s_labels],phrase_count=3)

100%|██████████| 410/410 [00:00<00:00, 13928.02it/s]


In [205]:
i = 83
print("Prediction: " + str(model.predict_proba(scaler.transform(s_feats[i-2]))[0][1]))
print("True label: %d" % s_labs[i-2])
phrases[i-2:i+1]

Prediction: 0.516284279539
True label: 0


/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[u'I SAID',
 u"SAID BECAUSE YOU HAVE YOUR TURN THERE'S A LINE I SAID I DIDN'T KNOW THERE WAS A LINE BUT WHAT",
 u'ARE WE WAITING FOR AFTER SEVEN YEARS OF OBAMA THIS IS NOT A TIME FOR PATIENCE THIS IS A TIME FOR URGENT ACTION BECAUSE IF WE GET THIS ELECTION WRONG']

In [139]:
Audio(s.voice_y[int(times[i][0]*s.sr):int((times[i][1]+2)*s.sr)],rate=s.sr)

In [140]:
Audio(s.crowd_y[int(times[i][0]*s.sr):int((times[i][1]+2)*s.sr)],rate=s.sr)

In [155]:
zip(s_feats[i-2],feature_names)

[(0.56992393212404913, 'internal_silence'),
 (0.18815326690673828, 'max_energy'),
 (186.86441040039062, 'max_pitch'),
 (0.054628688842058182, 'mean_energy'),
 (24.704828262329102, 'mean_pitch'),
 (0.0020158104598522186, 'min_energy'),
 (-1.0, 'min_pitch'),
 (0.18613745272159576, 'range_energy'),
 (187.86441040039062, 'range_pitch'),
 (0.041984330862760544, 'std_energy'),
 (33.330783843994141, 'std_pitch'),
 (0.20000000000000001, 'alliteration'),
 (0.47999999999999998, 'homogeneity'),
 (0.20000000000000001, 'plosive'),
 (0.22, 'rhyme'),
 (0.0, 'LIWC_achiev'),
 (1.0, 'LIWC_adj'),
 (0.0, 'LIWC_adverb'),
 (1.0, 'LIWC_affect'),
 (0.0, 'LIWC_affiliation'),
 (0.0, 'LIWC_anger'),
 (0.0, 'LIWC_anx'),
 (1.0, 'LIWC_article'),
 (0.0, 'LIWC_assent'),
 (1.0, 'LIWC_auxverb'),
 (1.0, 'LIWC_bio'),
 (0.0, 'LIWC_body'),
 (0.0, 'LIWC_cause'),
 (0.0, 'LIWC_certain'),
 (1.0, 'LIWC_cogproc'),
 (1.0, 'LIWC_compare'),
 (0.0, 'LIWC_conj'),
 (0.0, 'LIWC_death'),
 (0.0, 'LIWC_differ'),
 (0.0, 'LIWC_discrep'),
 (1

In [146]:
times[i][1]

323.74999899999995

In [144]:
s.applause_list.get_instances()

[(9.10125, 17.11128),
 (25.3767, 32.24907),
 (62.45501, 66.19302),
 (75.75862, 79.9842),
 (89.50337, 96.28287),
 (100.27628, 104.57151),
 (109.37753, 112.81371),
 (140.25677, 163.65999),
 (167.02652, 169.53401),
 (177.72978, 182.83762),
 (185.41476, 189.19921),
 (199.99534, 206.63553),
 (214.25087, 216.73514),
 (218.17462, 225.16308),
 (229.80657, 239.51148),
 (267.95289, 271.78377),
 (274.84848, 279.51519),
 (283.71755, 286.15539),
 (291.37932, 293.46889),
 (295.72099, 302.17545),
 (307.44581, 318.70629),
 (319.1242, 322.93187),
 (323.95344, 328.31832),
 (344.15264, 351.9305),
 (355.62208, 359.80122),
 (360.89244, 367.37012),
 (462.09742, 468.73762),
 (509.94864, 516.91388),
 (524.92391, 530.72828),
 (532.12133, 534.41986),
 (563.78996, 572.82156),
 (591.34911, 601.35584),
 (611.91979, 613.93971),
 (663.78763, 666.01651),
 (674.23549, 693.18095),
 (723.99054, 726.4516),
 (730.86292, 735.48319),
 (737.68885, 739.82486),
 (760.44198, 769.75219),
 (785.16859, 793.34114),
 (802.25665, 804

In [ ]:
featurizer.Featurizer(s).get_binary_labels()

In [46]:
len(feature_list)

412

In [109]:
s.load_librosa()
s.load_librosa_crowd()
s.load_librosa_voice()

True

In [114]:
s.applause_list.get_instances()

[(22.77613, 33.10781),
 (35.40632, 40.42124),
 (54.28195, 59.48261),
 (77.45277, 81.09788),
 (90.89556, 95.30684),
 (139.51249, 144.2256),
 (152.44451, 155.64849),
 (166.35164, 170.32179),
 (184.97188, 188.66343),
 (206.84254, 213.32015),
 (295.9968, 298.38818),
 (418.09635, 420.51095),
 (513.5425, 517.69839),
 (642.4447, 650.59396),
 (689.18104, 694.70675),
 (716.06662, 718.20261),
 (743.11473, 747.40992),
 (804.75654, 811.4199),
 (883.32374, 893.4929),
 (922.32873, 925.39341),
 (930.01364, 933.0551),
 (935.609, 942.80635),
 (1002.75331, 1008.27901),
 (1042.40838, 1045.2641),
 (1050.69694, 1058.61403),
 (1060.44819, 1068.31884),
 (1107.02201, 1111.24755),
 (1147.69864, 1152.92252),
 (1168.19948, 1173.6091),
 (1212.03366, 1214.68042),
 (1290.74016, 1294.87283),
 (1308.8264, 1311.47317),
 (1319.29738, 1322.45493),
 (1334.59755, 1339.03205),
 (1394.49808, 1401.95082),
 (1476.68716, 1479.47323),
 (1486.48485, 1495.56279),
 (1537.93429, 1547.96414),
 (1777.90785, 1788.611),
 (1963.5066, 19

In [72]:
np.max(s.get_rmse_at_times(times[i][0],times[i][1]))

0.11237981

In [82]:
def get_relative_rmse(s, start_time, end_time):
    mean_rmse = np.mean(s.rmse)
    return (np.mean(s.get_rmse_at_times(start_time, end_time)) / mean_rmse)

In [143]:
zip(np.array(all_feature_names)[np.argsort(coefs)],coefs[np.argsort(coefs)])

[('delta_1_min_energy', -5.3331412176435844),
 ('t_range_energy', -3.7987877529767142),
 ('delta_1_mean_energy', -3.2876116221160498),
 ('delta_1_max_energy', -2.6744090819347108),
 ('delta_2_mean_energy', -2.591703157930854),
 ('delta_2_min_energy', -1.5493668703809391),
 ('t-1_range_energy', -1.1400554998324293),
 ('t-2_max_energy', -1.067698892577204),
 ('t-2_min_energy', -0.84401951551697285),
 ('delta_2_max_energy', -0.63299062651404325),
 ('delta_1_applause_feature', -0.45032232456843896),
 ('t-1_max_energy', -0.43470826606265861),
 ('delta_1_homogeneity', -0.40243126603563639),
 ('t-2_mean_energy', -0.34277295451187861),
 ('t_LIWC_shehe', -0.24083157646188816),
 ('t_LIWC_function', -0.23940568192229625),
 ('t_LIWC_death', -0.22860996088981517),
 ('t-2_range_energy', -0.22368004587993517),
 ('t_LIWC_focuspast', -0.21149386259085526),
 ('t-1_LIWC_shehe', -0.20404594155490416),
 ('delta_1_LIWC_filler', -0.19736223456553884),
 ('t-2_LIWC_shehe', -0.19305332987880999),
 ('t_LIWC_see'

In [148]:
x=[('t_thank_you_feature', 0.3774252899630714),
('t-1_homogeneity', 0.38359345291463331),
('t_internal_silence', 0.51532797033195787),
('t-1_min_energy', 0.7053473548642436),
('t_homogeneity', 0.78602471894641057),
 ('t-1_applause_feature', 0.83723373483706343),
 ('delta_2_std_energy', 0.87771915384597043),
 ('delta_2_range_energy', 0.91637545394360398),
 ('t-2_applause_feature', 1.0031853932975721),
 ('t_applause_feature', 1.2875560594055036),
 ('t_max_energy', 2.2397008158732721),
 ('t-1_mean_energy', 2.2489302034200751),
 ('delta_1_std_energy', 2.3550216311442367),
 ('delta_1_range_energy', 2.6587322531427282),
 ('t-1_std_energy', 2.7222193509839596),
 ('t-2_std_energy', 3.599938504830182),
 ('t_mean_energy', 5.5365418255368963),
 ('t_min_energy', 6.0384885725075756)]

In [151]:
x

[('t_min_energy', 6.038488572507576),
 ('t_mean_energy', 5.536541825536896),
 ('t-2_std_energy', 3.599938504830182),
 ('t-1_std_energy', 2.7222193509839596),
 ('delta_1_range_energy', 2.658732253142728),
 ('delta_1_std_energy', 2.3550216311442367),
 ('t-1_mean_energy', 2.248930203420075),
 ('t_max_energy', 2.239700815873272),
 ('t_applause_feature', 1.2875560594055036),
 ('t-2_applause_feature', 1.003185393297572),
 ('delta_2_range_energy', 0.916375453943604),
 ('delta_2_std_energy', 0.8777191538459704),
 ('t-1_applause_feature', 0.8372337348370634),
 ('t_homogeneity', 0.7860247189464106),
 ('t-1_min_energy', 0.7053473548642436),
 ('t_internal_silence', 0.5153279703319579),
 ('t-1_homogeneity', 0.3835934529146333),
 ('t_thank_you_feature', 0.3774252899630714)]

In [137]:
np.argsort(coefs)

array([275,   7, 273, 271, 363, 365,  97, 181, 185, 361, 358,  91, 282,
       183,  77,  44,  32, 187,  41, 167, 309, 257,  75, 270, 134, 122,
        46,  84, 131, 359, 136,  82, 165,  30,  37, 399, 172, 127, 221,
       373, 224, 120, 296,  81, 360,  51, 262,  50,  60,  17, 193, 310,
       247,  47, 421,  80, 141, 449, 236, 125, 264, 158, 210, 174, 171,
        69, 423, 342, 319,  29, 225, 206, 355, 371, 308, 146, 230, 140,
        35, 107, 215, 372, 135, 427, 231, 294, 324, 430, 213, 298, 376,
        62, 139, 159, 255, 197,  56, 266, 313, 198, 176, 124, 386, 253,
        68, 436, 431, 425, 383, 119, 400, 346, 217, 398, 315, 414, 233,
       207,  86, 445, 248, 335, 433, 413, 108, 415, 212, 338, 202, 123,
       289,  53,  78, 387, 133, 379, 226, 137,  10, 157,  34, 214, 384,
       426, 261, 209, 393, 353, 250, 284, 116, 229, 150, 288,  31,  25,
        45,  33, 112, 375, 331, 104, 100, 447, 249, 304, 337, 432, 328,
       326, 388, 343, 306, 190, 305,  22, 285, 349,  52, 378, 41

In [127]:
# 3 phrases with no deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.647 +/- 0.011 (4941/7633) | Precision: 0.650 | Recall: 0.631 | F1: 0.640


(0.6473208437049652,
 0.65021691973969631,
 0.63105263157894742,
 0.64049145299145305)

In [84]:
s = speeches[81]

In [85]:
s.file_path

'donald_trump/donald_trump_33'

In [86]:
text = s.alignment.get_phrase_text()
times = s.alignment.get_phrase_times()

In [87]:
end_labels = [s.alignment.applause_follows(t[1]) for t in times]

In [88]:
zip(text,end_labels)

[(u'YOU EVERYBODY THANK YOU', True),
 (u'ARE GOING TO MAKE WE ARE', True),
 (u'AM THRILLED TO BE HERE TONIGHT BEAUTIFUL CITY MANNHEIM', True),
 (u'WILL TELL YOU WHAT WE WIN THE GREAT STATE OF PENNSYLVANIA I WENT TO SCHOOL IN PENNSYLVANIA',
  True),
 (u'ARE GOING TO WIN PENNSYLVANIA WE ARE GOING TO WIN BACK THE WHITE HOUSE AND WE ARE GOING TO BE SO HAPPY WE ARE GOING TO BE SO HAPPY AND WE GOING TO AGAIN BE PROUD OF OUR COUNTRY WE WILL BE PROUD WE GOING TO TAKE ON THE CORRUPT MEDIA THE POWERFUL LOBBYISTS AND THE SPECIAL INTERESTS THAT HAVE STOLEN YOUR JOBS HERE FACTORIES IN YOUR FUTURE THAT',
  False),
 (u'WHAT HAS HAPPENED WE ARE GOING TO STOP HILLARY CLINTON FROM CONTINUING TO RATE THE INDUSTRY FROM YOUR STATE FOR HER PROFIT',
  True),
 (u'CLINTON HAS COLLECTED MILLIONS OF DOLLARS FROM THE SAME GLOBAL CORPORATIONS SHIPPING YOUR JOBS YOUR DREAMS TO OTHER COUNTRIES YOU KNOW IT AND EVERYBODY ELSE KNOWS IT',
  False),
 (u'IS WHY CLINTON', False),
 (u'100 APPROVE TRANSPACIFIC', False),
 (u'

In [83]:
np.sum(end_labels)

7

In [147]:
x

NameError: name 'x' is not defined

In [60]:
import math